In [1]:
import pandas as pd
import numpy as np

import spacy
from spacy.tokenizer import Tokenizer

from sklearn.externals import joblib 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

nlp = spacy.load("en_core_web_lg")

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV

In [3]:
data = pd.read_csv('../DS_strain_data.csv', na_values='None')
# data = data.set_index('name')
print(data.shape)
data.head()

(2349, 11)


,Unnamed: 0,Id,Strain,Race,Flavors,Positive,Negative,Medical,Rating,Description,everything
0,0,0,100 Og,hybrid,"Earthy, Sweet, Citrus","Creative, Energetic, Tingly, Euphoric, Relaxed","Dizzy, Dry Mouth, Paranoid, Anxious","Depression, Stress, Fatigue",4.0,$100 OG is a 50/50 hybrid strain that packs a ...,"Earthy, Sweet, Citrus,Creative, Energetic, Tin..."
1,1,1,1024,sativa,"Spicy/Herbal, Sage, Woody","Uplifted, Happy, Relaxed, Energetic, Creative","Dizzy, Dry Mouth, Paranoid, Anxious","Depression, Stress, Fatigue",4.4,1024 is a sativa-dominant hybrid bred in Spain...,"Spicy/Herbal, Sage, Woody,Uplifted, Happy, Rel..."
2,2,2,13 Dawgs,hybrid,"Apricot, Citrus, Grapefruit","Tingly, Creative, Hungry, Relaxed, Uplifted","Dizzy, Dry Mouth, Paranoid, Anxious","Depression, Stress, Fatigue",4.2,13 Dawgs is a hybrid of G13 and Chemdawg genet...,"Apricot, Citrus, Grapefruit,Tingly, Creative, ..."
3,3,3,24K Gold,hybrid,"Citrus, Earthy, Orange","Happy, Relaxed, Euphoric, Uplifted, Talkative","Dizzy, Dry Mouth, Paranoid, Anxious","Depression, Stress, Fatigue",4.6,"Also known as Kosher Tangie, 24k Gold is a 60%...","Citrus, Earthy, Orange,Happy, Relaxed, Euphori..."
4,4,4,3 Bears Og,indica,"Sweet, Pungent, Earthy","Relaxed, Happy, Sleepy, Creative, Euphoric","Dizzy, Dry Mouth, Paranoid, Anxious","Depression, Stress, Fatigue",4.4,3 Bears OG by Mephisto Genetics is an autoflow...,"Sweet, Pungent, Earthy,Relaxed, Happy, Sleepy,..."


In [4]:
from gensim.utils import simple_preprocess

def tokenize(text):
    return [token for token in simple_preprocess(text)]

In [5]:
tfidf = TfidfVectorizer(stop_words = 'english',
                      # ngram_range=(1,2),
                      # max_df=.5,
                      # min_df=3,
                        tokenizer=tokenize)

dtm = tfidf.fit_transform(data['everything'])

dtm = pd.DataFrame(dtm.todense(), columns = tfidf.get_feature_names())
dtm.head()

,ammonia,anxious,appetite,apple,apricot,aroused,berry,blue,blueberry,butter,...,tar,tea,tingly,tobacco,tree,tropical,uplifted,vanilla,violet,woody
0,0.0,0.161923,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.448640,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
1,0.0,0.115515,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.156295,0.0,0.0,0.350248
2,0.0,0.103597,0.0,0.0,0.614224,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.287038,0.0,0.0,0.0,0.140171,0.0,0.0,0.000000
3,0.0,0.143521,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.194189,0.0,0.0,0.000000
4,0.0,0.177011,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000


In [6]:
from sklearn.neighbors import NearestNeighbors

nn = NearestNeighbors(n_neighbors=5, algorithm='kd_tree')
nn.fit(dtm)

NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=5, p=2, radius=1.0)

In [7]:
dtm2 = dtm.reset_index()
dtm2.shape

(2349, 88)

In [8]:
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
clf = NearestNeighbors()

In [12]:
import sklearn.pipeline as Pipeline

pipe = Pipeline([
                 #Vectorizer
                 ('vect', vect),
                 # Classifier
                 ('clf', clf)
                ])

TypeError: 'module' object is not callable

In [ ]:
#List Hyperparameters that we want to tune.
leaf_size = list(range(1,50))
n_neighbors = list(range(1,30))
p=[1,2]

#Convert to dictionary
hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)

#Create new KNN object
knn_2 = NearestNeighbors()

#Use GridSearch
clf = GridSearchCV(knn_2, hyperparameters, cv=2)

#Fit the model
best_model = clf.fit(dtm2.drop(columns='index'), dtm2.index)

#Print The value of best Hyperparameters
print('Best leaf_size:', best_model.best_estimator_.get_params()['leaf_size'])
print('Best p:', best_model.best_estimator_.get_params()['p'])
print('Best n_neighbors:', best_model.best_estimator_.get_params()['n_neighbors'])